In [1]:
import os
from os import listdir
from os.path import isfile, join
import pandas as pd
from subprocess import Popen,PIPE,STDOUT,call 
import json

In [2]:
def convert_resume_text_pdf(path):
    dirs = [os.path.join(path, o) for o in os.listdir(path) if os.path.isdir(os.path.join(path,o))]
    for x in dirs:
    #     x = x.replace(" ", "\ ")
        files = [f for f in listdir(x) if isfile(join(x, f))]
        candidates = [f for f in files if f.endswith('.txt') if f!="Success.txt" if f!="Job Description.txt"]
        for i in candidates:
            current_path = x+'/'+i
            current_path = current_path[:-4]
            current_path = current_path.replace(" ", "\ ")
            os.system('python txt2pdf.py -o '+ current_path +'.pdf '+current_path+'.txt')

In [3]:
def convert_description_text_pdf(path):
    dirs = [os.path.join(path, o) for o in os.listdir(path) if os.path.isdir(os.path.join(path,o))]
    for x in dirs:
    #     x = x.replace(" ", "\ ")
        files = [f for f in listdir(x) if isfile(join(x, f))]
        candidates = [f for f in files if f.endswith('.txt') if f!="Success.txt" if f=="Job Description.txt"]
        for i in candidates:
            current_path = x+'/'+i
            current_path = current_path[:-4]
            current_path = current_path.replace(" ", "\ ")
            os.system('python txt2pdf.py -o '+ current_path +'.pdf '+current_path+'.txt')

In [4]:
convert_description_text_pdf('data/')

In [5]:
def mine_resume_features(path):
    dirs = [os.path.join(path, o) for o in os.listdir(path) if os.path.isdir(os.path.join(path,o))]
    col_names =  ['candidate_name', 'job_title', 'jobs', 'qualification_info', 'job_category', 
                  'employers', 'current_employers', 'skills']
    data_  = pd.DataFrame(columns = col_names)
    for x in dirs:
    #     x = x.replace(" ", "\ ")
        job = (x.replace('data/', ''))
        files = [f for f in listdir(x) if isfile(join(x, f))]
        candidates = [f for f in files if f.endswith('.pdf') if f!="Success.txt" if f!="Job Description.txt"]
        for i in candidates:
            current_path = x+'/'+i
            current_path = current_path[:-4]
            current_path = current_path.replace(" ", "\ ")

            proc=Popen('cvscan parse --name '+ current_path, shell=True, stdout=PIPE, ) 
            data =proc.communicate()[0]
            data = data[1:]
            data = data[:-2]
            print (current_path)
            if len(data) == 0:
                pass
            else:
                data = json.loads(data.replace(r'\x3E', '\x3E'))
                data_.loc[len(data_)] = [i[:-4], job, ", ".join(data['jobs']), ", ".join(data['qualifications_info']), 
                                    data['job category'], ", ".join(data['employers']), 
                                         ", ".join(data['current_employers']), 
                                    ", ".join(data['skills'])]
    return data_

In [8]:
def mine_description_features(path):
    dirs = [os.path.join(path, o) for o in os.listdir(path) if os.path.isdir(os.path.join(path,o))]
    col_names =  ['job_title', 'skills']
    data_  = pd.DataFrame(columns = col_names)
    for x in dirs:
    #     x = x.replace(" ", "\ ")
        job = (x.replace('data/', ''))
        files = [f for f in listdir(x) if isfile(join(x, f))]
        candidates = [f for f in files if f.endswith('.pdf') if f!="Success.txt" if f=="Job Description.pdf"]
        for i in candidates:
            current_path = x+'/'+i
            current_path = current_path[:-4]
            current_path = current_path.replace(" ", "\ ")

            proc=Popen('cvscan parse --name '+ current_path, shell=True, stdout=PIPE, ) 
            data =proc.communicate()[0]
            data = data[1:]
            data = data[:-2]
            if len(data) == 0:
                pass
            else:
                data = json.loads(data.replace(r'\x3E', '\x3E'))
                data_.loc[len(data_)] = [job, ", ".join(data['skills'])]
    return data_

In [9]:
data = mine_description_features('data/')

In [11]:
data

,job_title,skills
0,exports administrator,"Administration , Communication , Marketing ..."
1,retail sales consultant,"Retail , Sales , Advantage , Art , BASIC ..."
2,financial reporting manager,"Financial Reporting , Management , Ability ..."
3,secretary,"Administration , Sales , Administration , ..."
4,receptionist,"Hospitality Management , Management , Time ..."
5,commercial analyst,"C , Management , Project Management , Adho..."
6,financial manager,"Finance , Financial Planning , Leadership ,..."
7,java developer,"Accounting , Communication , Databases , H..."
8,assistant accountant,"Management , Sales , Accounts , Accounts P..."
9,property sales,"Communication , Sales , Training , A , Ad..."


In [13]:
data.to_csv('descriptions.csv')